#### Cryptography
#### Task № 2 - DES
#### Ivan Rybin - ITMO JB SE MA 2021

In [1]:
import des_data

In [47]:
def xor_bytes(l, r):
    return bytearray([lb ^ rb for lb, rb in zip(l, r)])

def xor_bits(l, r):
    return "".join([str(int(lb) ^ int(rb)) for lb, rb in zip(l, r)])

def stobytes(s):
    return s.encode('utf-8')

def sto64(bits):
    while len(bits) % 64 != 0:
        bits += '0'
    return bits

def to8bits(bits):
    while len(bits) < 8:
        bits = '0' + bits
    return bits

def to4bits(bits):
    while len(bits) < 4:
        bits = '0' + bits
    return bits

def stobits(s):
    b = ""
    for c in stobytes(s):
        b += to8bits(bin(c)[2:])
    return b

def btoi(b):
    i = 0
    k = 0
    for c in reversed(b):
        i += int(c) * (2 ** k)
        k += 1
    return i

def bytestos(b):
    return b.decode('utf-8')

def bitstos(bits):
    s = []
    for b in split_to_blocks(bits, 8):
        s.append(btoi(b))
    return bytearray(s)

def perm(p, b):
    return ''.join([b[i] for i in p])

def split_to_blocks(b, block_size):
    blocks = []
    for i in range(1, int((len(b) + block_size) / block_size)):
        blocks.append(b[(i - 1) * block_size: i * block_size])
    return blocks

In [48]:
def ip_perm(b):
    return perm(des_data.ip_perm, b)

def ip_inv_perm(b):
    return perm(des_data.ip_inv_perm, b)

def p_perm(b):
    return perm(des_data.p_perm, b)

def g_perm(b):
    return perm(des_data.g_perm, b)

def h_perm(b):
    return perm(des_data.h_perm, b)

def calc_r(l, r_prev, key):
    return xor_bits(l, encrypt_func(r_prev, key))

In [49]:
def extend_perm(b):
    return perm(des_data.extend_perm, b)

def get_s_block_value(block, s_block):
    row = btoi(block[0] + block[-1]) # b1b6
    col = btoi(block[1:-1]) # b2b3b4b5
    return s_block[row * 16 + col]

def s_blocks_func(blocks):
    new_bits = ''
    for i in range(0, 8):
        s_block = des_data.s_blocks[i]
        new_block = get_s_block_value(blocks[i], s_block)
        new_bits += to4bits(bin(new_block)[2:])
    return new_bits


def encrypt_func(r, key):
    # r: 32 bites -> r_extend: 48 bites
    r_extend = extend_perm(r)    
    r_xor_key = xor_bits(r_extend, key) # key: 48 bites
    blocks = split_to_blocks(r_xor_key, 6) # 8 blocks x 6 bites
    new_r = s_blocks_func(blocks)
    new_r = p_perm(new_r)
    return new_r

In [50]:
def key64_56(key):
    bits = stobits(key)
    return g_perm(bits)


def cyclic_offset(b, offset):
    new_b = ''
    for i in range(offset, offset + len(b)):
        new_b += b[i % len(b)]
    return new_b


def get_all_keys(key):
    c = []
    d = []
    keys = []
    key_56 = key64_56(key)
    c.append(key_56[0:28])
    d.append(key_56[28:])
    for i in range(1, 17): # from 1 to 16
        offset = des_data.key_offsets[i - 1]
        c_i = cyclic_offset(c[i - 1], offset)
        d_i = cyclic_offset(d[i - 1], offset)
        key_i = h_perm(c_i + d_i)
        keys.append(key_i)
        c.append(c_i); d.append(d_i)
        
    return keys

In [51]:
def des_encrypt_round(block, key):
    l = block[0:32]
    r = block[32:]
    new_l = r; new_r = xor_bits(l, encrypt_func(r, key))
    return new_l, new_r


def des_decrypt_round(block, key):
    l = block[0:32]
    r = block[32:]
    new_l = xor_bits(r, encrypt_func(l, key)); new_r = l
    return new_l, new_r


def des_encrypt(s, key):
    sbits = sto64(stobits(s)) # bits string
    sblocks = split_to_blocks(sbits, 64) # N blocks x 64 bits
    keys = get_all_keys(key) # all keys k_i, i=0..16
    
    encrypt_blocks = []
    for i in range(0, len(sblocks)):
        sblock = sblocks[i]
        # ip permutation
        sblock = ip_perm(sblock)
        # 15 rounds
        for i in range(0, 16): # from 1 to 16
            key = keys[i]
            l, r = des_encrypt_round(sblock, key)
            sblock = l + r
        sblock = sblock[32:] + sblock[0:32]
        # ip inv perm
        sblock = ip_inv_perm(sblock)
        encrypt_blocks.append(sblock)
    return ''.join(encrypt_blocks)


def des_decrypt(e, key):
    eblocks = split_to_blocks(e, 64) # N blocks x 64 bits
    keys = get_all_keys(key) # all keys k_i, i=0..16
    keys = [item for item in reversed(keys)]
    
    decrypt_blocks = []
    for i in range(0, len(eblocks)):
        eblock = eblocks[i]
        eblock = ip_perm(eblock)
        for i in range(0, 16): # from 1 to 16
            key = keys[i]
            l, r = des_encrypt_round(eblock, key)
            eblock = l + r
        # ip inv perm
        eblock = eblock[32:] + eblock[0:32]
        eblock = ip_inv_perm(eblock)
        decrypt_blocks.append(eblock)
    return ''.join(decrypt_blocks)

In [52]:
key = 'crypto42'
message = 'это сообщение было зашифровано DES алгоритмом с ключом crypto42'
message_bits = sto64(stobits(message))
print(message_bits)

11010001100011011101000110000010110100001011111000100000110100011000000111010000101111101101000010111110110100001011000111010001100010011101000010110101110100001011110111010000101110001101000010110101001000001101000010110001110100011000101111010000101110111101000010111110001000001101000010110111110100001011000011010001100010001101000010111000110100011000010011010001100000001101000010111110110100001011001011010000101100001101000010111101110100001011111000100000010001000100010101010011001000001101000010110000110100001011101111010000101100111101000010111110110100011000000011010000101110001101000110000010110100001011110011010000101111101101000010111100001000001101000110000001001000001101000010111010110100001011101111010001100011101101000110000111110100001011111011010000101111000010000001100011011100100111100101110000011101000110111100110100001100100000000000000000000000000000000000000000


In [53]:
encrypted = des_encrypt(message, key)
print(f'{encrypted}')

10011001111000111010110010110010110111010100110110001010100011011101010011011101001111111111100100111111110110110001111001100010010111010001010101100111011000011011100010100000101101000110100111000110111010001101011110000111010011100101001110101111001001101111010011011101000111101111001000101010000100100101111111101001000111011101010000111011100010101000100010011001001100010011111101011111111000010100101011100011001000100000010011001011111000010011110111101110010101101010010000111001100100111010111001001110001011011011110111101010010000111100100100000111011010101100100101100000011000100110111010010110001011011001001100100111100111001100101111100001110101101010001001011000010000111010111100100111010011011111010101101111101011000000101111001101011100111001011101011001001101100110100110110110110100100011011110111101000100000010001011010010101101100100010110000000110010000110011110110010


In [54]:
decrypted = des_decrypt(encrypted, key)
print(f'{decrypted}')

11010001100011011101000110000010110100001011111000100000110100011000000111010000101111101101000010111110110100001011000111010001100010011101000010110101110100001011110111010000101110001101000010110101001000001101000010110001110100011000101111010000101110111101000010111110001000001101000010110111110100001011000011010001100010001101000010111000110100011000010011010001100000001101000010111110110100001011001011010000101100001101000010111101110100001011111000100000010001000100010101010011001000001101000010110000110100001011101111010000101100111101000010111110110100011000000011010000101110001101000110000010110100001011110011010000101111101101000010111100001000001101000110000001001000001101000010111010110100001011101111010001100011101101000110000111110100001011111011010000101111000010000001100011011100100111100101110000011101000110111100110100001100100000000000000000000000000000000000000000


In [55]:
decrypted == message_bits

True

In [57]:
bitstos(decrypted).decode('utf-8')

'это сообщение было зашифровано DES алгоритмом с ключом crypto42\x00\x00\x00\x00\x00'